In [1]:
import getpass
import json
import uuid

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import (check_premises,
                                   execute_query,
                                   execute_query_pipe,
                                   apply_rule,
                                   get_rule_parameters)

In [2]:
LOCAL_TOKEN = True 
TOKEN_PATH = 'token.txt'
CONFIG_PATH = "config.yaml"

def allocate_forge_session(org, project):
    ENDPOINT = "https://bbp.epfl.ch/nexus/v1"  

    if LOCAL_TOKEN: 
        with open(TOKEN_PATH) as f:
            TOKEN = f.read()
    else: 
        TOKEN = getpass.getpass()
        
    searchendpoints = {
        "sparql": {
           "endpoint": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
       }
    }
    
    return KnowledgeGraphForge(
        CONFIG_PATH,
        endpoint=ENDPOINT,
        token=TOKEN, 
        bucket=f"{org}/{project}", 
        searchendpoints=searchendpoints,
        debug=True
    )

In [3]:
forge = allocate_forge_session("bbp", "atlas")
res = forge.search({
    "type": "<https://neuroshapes.org/NeuronMorphology>",
    "contribution": {
        "agent": {"id":"<https://bbp.epfl.ch/neurosciencegraph/data/7c47aa15-9fc6-42ec-9871-d233c9c29028>"}
    }
}, cross_bucket=True)

len(res)

158

# ForgeSearchQuery

In [12]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

my_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "TEST",
    "name": "Test ForgeQuerySearch",
     "searchQuery": {
        "type": "ForgeSearchQuery",
        "hasParameter": [
            {
                "type": "uri",
                "description": "test filter",
                "name": "TypeQueryParameter"
            },
             {
                "type": "uri",
                "description": "test filter",
                "name": "ContributionQueryParameter"
            },
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            "org": "bbp",
            "project": "atlas"
        },
        "pattern": {
            "type": "$TypeQueryParameter",
            "contribution": {
                "agent": {"id":"$ContributionQueryParameter"}
            }
        },
        "cross_bucket":True
    
    }
}

input_filters = {
    "TypeQueryParameter": "<https://neuroshapes.org/NeuronMorphology>",
    "ContributionQueryParameter": "<https://bbp.epfl.ch/neurosciencegraph/data/7c47aa15-9fc6-42ec-9871-d233c9c29028>"
}

res = apply_rule(allocate_forge_session, my_rule, input_filters)
len(res)

Submitted query:
   PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
   PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
   PREFIX commonshapes: <https://neuroshapes.org/commons/>
   PREFIX datashapes: <https://neuroshapes.org/dash/>
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
   PREFIX oa: <http://www.w3.org/ns/oa#>
   PREFIX obo: <http://purl.obolibrary.org/obo/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFI

158

# ForgeSearchPremise

In [5]:
rule_id2 = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"

my_rule2 = {
    "id": rule_id2,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "TEST",
    "name": "Test ForgeQuerySearch",
    "premise": {
        "type": "ForgeSearchPremise",
        "hasParameter": [
            {
                "type": "uri",
                "description": "test filter",
                "name": "TypeQueryParameter"
            },
            {
                "type": "uri",
                "description": "test filter",
                "name": "ContributionQueryParameter"
            },
            {
                "type": "path",
                "description": "test",
                "name": "AgeQueryParameter"
            }
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            "org": "bbp",
            "project": "atlas"
        },
        "targetParameter": "AgeQueryParameter",  
        "targetPath": "subject.age.value", 
        "pattern": {
            "type": "$TypeQueryParameter",
            "contribution": {
                "agent": {"id": "$ContributionQueryParameter"}
            }
        }

    }
}

input_filters2 = {
    "TypeQueryParameter": "<https://neuroshapes.org/NeuronMorphology>",
    "ContributionQueryParameter": "<https://bbp.epfl.ch/neurosciencegraph/data/7c47aa15-9fc6-42ec-9871-d233c9c29028>",
    "AgeQueryParameter": 60
}

res2 = check_premises(allocate_forge_session, my_rule2, input_filters2)
res2

True